In [16]:
#use tensorflow to build a neural network to predict the price of a house
#select all the features and the price
#split the data into training and testing data
#use the california housing data set
#use the mean squared error as the loss function
#use the adam optimizer
#use the mean absolute error as the metric

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_california_housing
from tqdm.keras import TqdmCallback



/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#load the data
housing = fetch_california_housing()
#convert the data to a pandas dataframe
df = pd.DataFrame(housing.data, columns=housing.feature_names)
#add the price column
df['Price'] = housing.target
#select all the features and the price
X = df.drop('Price', axis=1)
y = df['Price']
#split the data into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [3]:
#build the model
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[8]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
])


2022-12-14 14:20:23.262944: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-12-14 14:20:23.269624: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-14 14:20:23.269677: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (codespaces-bb7267): /proc/driver/nvidia/version does not exist
2022-12-14 14:20:23.270217: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
#compile the model
model.compile(loss='mean_squared_error',
                optimizer='adam',
                metrics=['mean_absolute_error'])

#train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)

#evaluate the model
model.evaluate(X_test, y_test, verbose=2)



129/129 - 0s - loss: 0.2687 - mean_absolute_error: 0.3562 - 210ms/epoch - 2ms/step


[0.2686731517314911, 0.35616806149482727]

In [5]:
#predict the price of a house using features in an array
#the features are in the same order as the columns in the dataframe

#features = [longitude, latitude, housing_median_age, total_rooms, total_bedrooms, population, households, median_income]
features = [-122.23, 37.88, 41.0, 880.0, 129.0, 322.0, 126.0, 8.3252]
features = np.array(features)
features = features.reshape(1, -1)
features = scaler.transform(features)
prediction = model.predict(features)
print(prediction)


1/1 [==============================] - 0s 78ms/step
[[-244.3643]]


/home/codespace/.local/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


## Model improvement

In [20]:
#use grid search to find the best hyperparameters

from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV

def build_model(optimizer='adam'):
    model = keras.Sequential([
        layers.Dense(64, activation='relu', input_shape=[8]),
        layers.Dense(64, activation='relu'),
        layers.Dense(1)
    ])
    model.compile(loss='mean_squared_error',
                optimizer=optimizer,
                metrics=['mean_absolute_error'])
    return model

model = KerasRegressor(build_fn=build_model, verbose=0)
params = {'batch_size': [32, 64, 128],
            'epochs': [100, 200, 300],
            'optimizer': ['adam', 'rmsprop']}
grid = GridSearchCV(estimator=model, param_grid=params, cv=3)
grid_result = grid.fit(X_train, y_train, callbacks=[TqdmCallback(verbose=0)])
print(grid_result.best_score_)
print(grid_result.best_params_)

/tmp/ipykernel_15542/402695260.py:17: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasRegressor(build_fn=build_model, verbose=0)




















































































































































































































































































































100%|██████████| 100/100 [00:34<00:00,  2.88epoch/s, loss=0.21, mean_absolute_error=0.317]
